In [26]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from webdriver_manager.chrome import ChromeDriverManager
import time
from time import sleep
import random
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
import re

In [3]:
# Setting
# 크롬 드라이버 다운로드 및 자동 설정
chrome_driver_path = ChromeDriverManager().install()

# 브라우저 꺼짐 방지 옵션
chrome_options = Options()
chrome_options.add_experimental_option("detach", True)

# 불필요한 에러 메시지 삭제
chrome_options.add_experimental_option("excludeSwitches", ["enable-logging"])

# 저장할 파일의 경로 설정
path = "E:/MOON/capstone_data"  # E 드라이브의 MOON 폴더 경로. 데이터를 저장해줄 경로

In [4]:
data = pd.read_csv(path + "/restaurant_자양동_음식점_naver.csv")
urls = data['url']

url = urls[0]
driver = webdriver.Chrome(service=Service(chrome_driver_path), options=chrome_options)
driver.get(url)
driver.maximize_window()
time.sleep(5)

In [34]:
urls = data['url']
urls

0      https://map.naver.com/p/search/%EC%9E%90%EC%96...
1      https://map.naver.com/p/search/%EC%9E%90%EC%96...
2      https://map.naver.com/p/search/%EC%9E%90%EC%96...
3      https://map.naver.com/p/search/%EC%9E%90%EC%96...
4      https://map.naver.com/p/search/%EC%9E%90%EC%96...
                             ...                        
299    https://map.naver.com/p/search/%EC%9E%90%EC%96...
300    https://map.naver.com/p/search/%EC%9E%90%EC%96...
301    https://map.naver.com/p/search/%EC%9E%90%EC%96...
302    https://map.naver.com/p/search/%EC%9E%90%EC%96...
303    https://map.naver.com/p/search/%EC%9E%90%EC%96...
Name: url, Length: 304, dtype: object

In [27]:
def remove_emoji(text):
    # 이모지 패턴 정규 표현식
    emoji_pattern = re.compile("["
                               u"\U0001F600-\U0001F64F"  # 스마일 이모티콘
                               u"\U0001F300-\U0001F5FF"  # 이모티콘 및 기호
                               u"\U0001F680-\U0001F6FF"  # 기호
                               u"\U0001F1E0-\U0001F1FF"  # 국기 표시 (1)
                               "]+", flags=re.UNICODE)
    # 이모지 제거
    return emoji_pattern.sub(r'', text)

def find_show_more():
    try:
        # '더보기' 버튼을 찾음
        show_more_button = WebDriverWait(driver, 2).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, '.TeItc'))
        )
        # 버튼 클릭
        show_more_button.click()
        time.sleep(2)

        return 1
    except:
        # 더 이상 '더보기' 버튼이 없으면 반복 종료
        return -1

def crawl_review(driver, rst_name):
    user = driver.find_element(By.CSS_SELECTOR, ".P9EZi").text                                  # 유저 이름

    time.sleep(1)

    # 리뷰 텍스트 전체보기 클릭
    try:
        more_button = driver.find_element(By.CSS_SELECTOR, ".WPk67")
        more_button.click()
    except:
        pass
    
    # 리뷰가 적혀있지 않은 경우, None으로 처리
    try:
        review_text = driver.find_element(By.CSS_SELECTOR, ".zPfVt").text.replace("\n", " ")    # 리뷰 텍스트
        review_text = remove_emoji(review_text)
    except:
        review_text = None
    
    u_rst_tag = []
    try:
        tags = driver.find_elements(By.CSS_SELECTOR, ".sIv5s")                                  # 유저가 리뷰에 남긴 태그
        for tag in tags:
            u_rst_tag.append(tag.text)
    except:
        tags = None
    
    try:
        date = driver.find_element(By.CSS_SELECTOR, ".CKUdu time").text                         # 리뷰 남긴 날짜
    except:
        date = None

    result = [rst_name, user, review_text, u_rst_tag, None, date, 0]                            # 리스트로 저장

    return result

def crawl_review_info(rst_name):
    tabs = driver.find_elements(By.CSS_SELECTOR, "._tab-menu")
    for tab in tabs:
        if tab.text == '리뷰':
            review_button = tab
            break                                                                                   # 리뷰 정보 수집
    review_button.click()
    time.sleep(2)

    tag = 1
    while tag != -1:
        tag = find_show_more()                                                                  # 끝까지 '더보기' 버튼 클릭
    print("\n리뷰 전체 로드 완료\n")
    
    results = []                                                                                # 레스토랑 하나 당 결과를 담을 리스트
    reviews = driver.find_elements(By.CSS_SELECTOR, ".owAeM")

    print("Review Info\n")
    print(f"해당 식당의 리뷰 개수는 {len(reviews)}입니다\n")
    for i, review in enumerate(reviews):
        result = crawl_review(review, rst_name)
        results.append(result)

        if i % 30 == 0:
            print(f"{i}번째 리뷰 수집 완료입니다")
            print(result)
            print()

        time.sleep(1)
    print("리뷰 전체 수집 완료")
    
    return results  


In [16]:
def switch_to_search_iframe(driver):
    # 지도가 아닌 검색 탭의 iframe으로 전환
    iframe = driver.find_element(By.CSS_SELECTOR, "iframe#searchIframe")
    driver.switch_to.frame(iframe)
    time.sleep(3)
    print("iframe 전환")

def switch_to_info_iframe(driver):
    # 전체 페이지로 복귀하도록 iframe에서 벗어나기 -> 그래야만 새로 생긴 iframe 탭에 접근할 수 있음
    driver.switch_to.default_content()
    time.sleep(0.5)

    # 새로 생긴 식당 탭 iframe으로 전환
    iframe = driver.find_element(By.CSS_SELECTOR, "#entryIframe")
    driver.switch_to.frame(iframe)
    time.sleep(1)

In [30]:
results = []

# switch_to_search_iframe(driver)
# urls 리스트에서 첫 번째 요소는 이미 열렸으므로 두 번째 요소부터 처리
for url in urls[0:2]:
    # 식당 정보 iframe 전환
    switch_to_info_iframe(driver)

    # 이름 정보 수집
    name = driver.find_element(By.CSS_SELECTOR, ".Fc1rA").text
    
    print("===================================================================================================")
    print("Restaurant Name\n")
    print(name)
    print()

    # 리뷰 정보 수집
    result = crawl_review_info(name)
    results.extend(result)             


    driver.get(url)
    time.sleep(3.5)
# 마지막 탭 닫기
# driver.close()

Restaurant Name

고이장


리뷰 전체 로드 완료

Review Info

해당 식당의 리뷰 개수는 10입니다

0번째 리뷰 수집 완료입니다
['고이장', 'modejoos', '나만 알고 싶은 힙한 분위기 술집 고이장️ 골목길 속에서 귀여운 고추간판을 찾아보세요❣️한우육회를 가성비 넘치는 가격에 드실 수 있는 곳이에요🥹 특히 사장님께서 직접 만드신 한국전통 고추장을 이용한 고추마요소스에 육회찍먹🥩 꼭 해봐야해요! 주종이 다양해서 뭘 먹을지 몰랐는데 사장님이 추천해주신 전통주랑 넘 잘 어울려서 한 병 뚝딱 했습니다 펫프렌들리매장이니까 강아지 친구들도 놀러오세요✨', ['인테리어가 멋져요', '음식이 맛있어요', '술이 다양해요', '가성비가 좋아요', '특별한 메뉴가 있어요'], None, '3.21.목', 0]

1번째 리뷰 수집 완료입니다
['고이장', '아차산도토리', '남편이랑 화이트데이에 데이트한 고이장\U0001fa77 입구부터 심상치 않았는데 맛도 분위기도 너무 좋았어요 육회가 너무 신선해서 입에서 살살 녹고 육감 주먹밥은 고소하고 육회의 맛을 잘 살려주면서 조합이 너무 좋았어요\U0001fa77 우리가 선택한 술은 해방주!! 신선한 향을 담아 순하고 상큼하다고 해서 선택했는데 정말 순하고 상큼해서 술이 술술 들어갔네요! 건대에 이렇게 분위기 좋은 술집이 있는지 몰랐는데 앞으로 종종 많이 놀러갈 것 같아요\U0001fa77 맛도 최고 분위기도 최고 였습니당\U0001fa77', ['음식이 맛있어요', '특별한 메뉴가 있어요', '가성비가 좋아요', '건강한 맛이에요', '인테리어가 멋져요'], None, '3.14.목', 0]

2번째 리뷰 수집 완료입니다
['고이장', 'sunday09', '육사시미 맛있습니다  찌개는 일반 고추장찌개보다 텁텁하지않고 고기가 많이들어있네요  저는 호인데 남편은 쏘쏘라고. 조금 아쉬워서 감태주먹밥 시켰는데 비쥬얼미쳤 ㅠㅠ  밥이 조금 고슬고슬하믄 좋겠어요 ㅎㅎ 감태위 육회를 넉넉히 주셔서 좋네요.  육

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"css selector","selector":"#entryIframe"}
  (Session info: chrome=123.0.6312.86)
Stacktrace:
	GetHandleVerifier [0x010F4CC3+225091]
	(No symbol) [0x01024E11]
	(No symbol) [0x00EC9A7A]
	(No symbol) [0x00F0175B]
	(No symbol) [0x00F0188B]
	(No symbol) [0x00F37882]
	(No symbol) [0x00F1F5A4]
	(No symbol) [0x00F35CB0]
	(No symbol) [0x00F1F2F6]
	(No symbol) [0x00EF79B9]
	(No symbol) [0x00EF879D]
	sqlite3_dbdata_init [0x01569A63+4064547]
	sqlite3_dbdata_init [0x0157106A+4094762]
	sqlite3_dbdata_init [0x0156B968+4072488]
	sqlite3_dbdata_init [0x0126C9C9+930953]
	(No symbol) [0x010307E4]
	(No symbol) [0x0102AD08]
	(No symbol) [0x0102AE31]
	(No symbol) [0x0101CAA0]
	BaseThreadInitThunk [0x76C17BA9+25]
	RtlInitializeExceptionChain [0x77C4BDAB+107]
	RtlClearBits [0x77C4BD2F+191]


In [29]:
 # 리뷰 스키마
rev_cols = ['rst_name', 'user_name', 'review_text', 'u_rst_tag', 'ate_menus', 'date', 'platform']
df_review = pd.DataFrame(results, columns=rev_cols)
df_review.to_csv(path + f"/review_test2.csv", encoding='utf-8-sig')